In [104]:
import sys
import json
import numpy as np
params = {}
default = {'training_folder': '../../scenarios/houston/', 'params': params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../../scenarios/houston/', 'params': {}}


In [105]:
def set_index(df):
    if 'index' in df.columns:
        df = df.set_index('index')
    else:
        df.index.name='index'
    return df

In [106]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS
print('num cores:',num_cores)

io_engine= 'pyogrio' if on_lambda else 'pyogrio' #or fiona

num cores: 8


In [107]:
base_folder = argv['training_folder']
input_folder = os.path.join(base_folder,'inputs/')
pt_folder  = os.path.join(input_folder,'pt/')
road_folder = os.path.join(input_folder,'road/')
od_folder =  os.path.join(input_folder,'od/')
params_folder = input_folder if on_lambda else '../../inputs/'

output_folder = os.path.join(base_folder,'outputs/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
model_folder = os.path.join(input_folder, 'model/')

In [108]:
var = excel.read_var(file=os.path.join(params_folder,'parameters.xlsx'), scenario='base', return_ancestry=False)
if 'params' in argv.keys():
    var.update(pd.DataFrame.from_dict(argv['params'], orient="index").stack())
    
cutoff = var['pathfinder']['cutoff']   

# inputs

In [109]:
sm = stepmodel.read_zippedpickles(model_folder + 'distribution')

road_nodes: 100%|███████████████████████████████| 28/28 [00:01<00:00, 16.44it/s]


# od_set

In [110]:
v = sm.volumes.set_index(['origin', 'destination']).sum(axis=1)
od_set = set(v.loc[v>0].index)
len(od_set)

10000

# PT pathfinder

In [111]:
sm.step_pt_pathfinder(
    broken_routes=False,
    broken_modes=False,
    keep_pathfinder=True,
    mode_column='route_type',
    route_column='route_id',
    speedup=True,
    walk_on_road=True,
    path_analysis=False,
    od_set=od_set,
    engine='b', # b is faster!
)
ppf = sm.publicpathfinder

start publicpathfinder
path_analysis


# Pruning

### Set broken routes and modes

In [112]:
broken_bus = []
broken_routes = []
print('Broken Routes:', broken_routes)

Broken Routes: []


In [113]:
mode_combinations = []
if var.get('pathfinder',{}).get('walk_pathfinder'):
    mode_combinations.append(set(sm.links['route_type'].unique()))

for mode in sm.links['route_type'].unique():
    mode_combinations.append({mode})
    
print('Broken Modes Combinaison:', mode_combinations)

Broken Modes Combinaison: [{'bus'}, {'tram'}]


In [114]:
ppf.combinations = dict()
ppf.combinations['route_id'] = [{route} for route in broken_routes]
ppf.combinations['route_type'] = mode_combinations

ppf.build_route_id_sets(first_and_last_only=True)
ppf.build_route_type_sets(first_and_last_only=False)
ppf.build_od_sets()

ppf.splitted_od_sets['route_id'] = {
    c : efficient_od_sets(od_set, factor=0.6, verbose=True) 
    for c, od_set in ppf.od_sets['route_id'].items()
}


ppf.broken_route_paths = pd.DataFrame()
ppf.broken_mode_paths = pd.DataFrame()

if var.get('pathfinder',{}).get('route_breaker'):
    ppf.find_broken_combination_paths(column='route_id', cutoff=cutoff, build_shortcuts=False, prune=False)
    ppf.broken_route_paths = ppf.broken_combination_paths
    ppf.broken_route_paths['pathfinder_session'] = 'route_breaker' 
    ppf.broken_route_paths['broken_route'] = ppf.broken_route_paths['broken_route_id'].apply(
        lambda s: list(s)[0]
    )
    
if var.get('pathfinder',{}).get('mode_breaker'):
    ppf.find_broken_combination_paths(column='route_type', cutoff=cutoff, build_shortcuts=False, prune=False)
    ppf.broken_mode_paths = ppf.broken_combination_paths
    ppf.broken_mode_paths['pathfinder_session'] = 'mode_breaker'
    ppf.broken_mode_paths['broken_modes'] = ppf.broken_mode_paths['broken_route_type'].apply(set)

route_type {'tram'}: 100%|████████████████████████| 2/2 [00:00<00:00,  2.40it/s]


In [115]:
pt_los = pd.concat([
    ppf.best_paths,
    ppf.broken_mode_paths, 
    ppf.broken_route_paths, 
    ]
)
pt_los['path'] = [tuple(p) for p in pt_los['path']]
pt_los = pt_los.drop_duplicates(subset=['path'])

print(len(pt_los))
sm.pt_los = pt_los
pt_los.set_index(['origin', 'destination'])['pathfinder_session'].value_counts()


10027


best_path       10000
mode_breaker       27
Name: pathfinder_session, dtype: int64

In [116]:

sm.step_road_pathfinder(method='aon', access_time='time',time_column='time',od_set=od_set,path_analysis=True)

In [117]:
sm.analysis_pt_los(walk_on_road=True)
sm.analysis_pt_route_type(hierarchy=[ 'car','rail', 'subway', 'tram', 'bus', 'walk'])

path_analysis: 100%|███████████████████| 10027/10027 [00:00<00:00, 18772.30it/s]


In [118]:
sm.to_zippedpickles(model_folder +'los', omitted_attributes=['publicpathfinder'])

publicpathfinder: 100%|█████████████████████████| 29/29 [00:04<00:00,  6.16it/s]
